In [ ]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import datetime as dt
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from keys import (username, password)

In [ ]:
# Reviews data covers October 2010 through April 2019
# On first run of scraping, we discovered that
## Crowd Level began to include a prediction starting on August 12, 2014.
## Start collecting data in September to grab full month of data
dates = []

start_date = dt.date(2014, 9, 1)
end_date = dt.date(2019, 4, 30)
delta = dt.timedelta(days=1)

while start_date <= end_date:
    dates.append(start_date.strftime("%Y-%m-%d"))
    start_date += delta
    
print(dates)

In [ ]:
## Reworking code - discovered that later years have a prediction before the actual crowd level. 

# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

browser.visit('https://touringplans.com/login')
UserFill = browser.find_by_id('user_session_login')
PassFill = browser.find_by_id('user_session_password')
UserFill.fill(username)
PassFill.fill(password)
button = browser.find_by_id('Submit')
button.click()

# https://touringplans.com/disneyland-resort/crowd-calendar/date/2010-10-13
url = "https://touringplans.com/disneyland-resort/crowd-calendar/date/"

date = []
max_temp = []
min_temp = []
crowd_level = [] 

for x in dates:
    try:
        browser.visit(url + x)
        html = browser.html
        soup = bs(html, "html.parser")
        weathercontainer = soup.find('table', class_="weather")
        weather = weathercontainer.find_all('td')
        crowds = soup.find_all('span', class_="crowd_cal_crowd_level")
        crowdstrong = soup.find_all('strong')

        # fill the variables for the current date
        # hold temporarily to ensure we get all data points, otherwise whole row will error and drop
        temp_max_temp = weather[1].text.split(':')[1].split(' ')[1].split('°')[0]
        temp_min_temp = weather[1].text.split(':')[2].replace('\n\t', '').replace(' ', '').split('°')[0]
        temp_crowd_level = crowdstrong[1].text.strip().split(' ')[0]
        print(f"{x} grabbed")
        # fill the variables for the current date
        date.append(x)
        max_temp.append(temp_max_temp)
        min_temp.append(temp_min_temp)
        crowd_level.append(temp_crowd_level)
        print(f"data appended")
#         dt.time.sleep(1)
    except (TypeError, NameError, IndexError) as err:
        print(f"{x}: {err}")
    

In [ ]:
scrapedData = {
    'date' : date,
    'max_temp' : max_temp,
    'min_temp' : min_temp,
    'crowd_level': crowd_level
}
scrapedf = pd.DataFrame(scrapedData)
scrapedf.head()

In [ ]:
scrapedf.to_csv('scraped_data.csv', header=True, index=False)